In [5]:
from Crypto.Cipher import AES
from Crypto import Random
from tkinter import * 
from PIL import ImageTk, Image
from tkinter import ttk
from tkinter.ttk import *

from tkinter.filedialog import askopenfile 
import io
import os
import math 
import hashlib


format = "png"
IV = Random.new().read(AES.block_size)
path=""
filename=''

root = Tk() 
root.geometry('500x180')
root.title('Image Encryption & Decryption')

def pad(data):
    return data + b"\x00"*(16-len(data)%16) 
 
def convert_to_RGB(data):
    r, g, b = tuple(map(lambda d: [data[i] for i in range(0,len(data)) if i % 3 == d], [0, 1, 2]))
    pixels = tuple(zip(r,g,b))
    return pixels

def open_file():
    global filename
    filename = filedialog.askopenfilename()
    path=filename
    filename = os.path.basename(filename)
    # print(filename)
    show()
    
def enc_click():
    global enc_key
    enc_key = name.get()
    enc_key = hashlib.sha256(enc_key.encode('utf-8')).digest()#256 bit encryption key
    encrypt_file()
    
def encrypt_file():
    global filename
    im = Image.open(filename)
    data = im.convert("RGB").tobytes() 
    original = len(data) 
    new = convert_to_RGB(aes_cbc_encrypt(enc_key, pad(data))[:original]) 
    im2 = Image.new(im.mode, im.size)
    im2.putdata(new)

    im2.save(filename.split('.')[0]+"_Encrypted.png", format)
    filename=filename.split('.')[0]+"_Encrypted.png"
    show()
    
    
    
def get_enc_key():
    lbl = ttk.Label(root, text = "Enter the key:").place(x=150,y=100)
    global name
    name=StringVar()
    nameEntered = ttk.Entry(root, width = 12, textvariable = name).place(x=250,y=100) 
    button = ttk.Button(root, text = "submit", command = enc_click).place(x=200,y=130)
    
    
def dec_click():
    global dec_key
    dec_key = key1.get()   
    dec_key = hashlib.sha256(dec_key.encode('utf-8')).digest()
    decrypt_file()
    
def decrypt_file():
    global filename
    im = Image.open(filename)
    data = im.convert("RGB").tobytes() 
    original = len(data) 
    new = convert_to_RGB(aes_cbc_decrypt(dec_key, pad(data))[:original]) 
    im2 = Image.new(im.mode, im.size)
    im2.putdata(new)

    im2.save(filename.split('.')[0]+"_Decrypted.png", format)
    filename = filename.split('.')[0]+"_Decrypted.png"
    show()
    
       
def get_dec_key():
    lbl = ttk.Label(root, text = "Enter the key:").place(x=150,y=100)
    global key1
    key1 = StringVar()   
    nameEntered = ttk.Entry(root, width = 12, textvariable = key1).place(x=250,y=100)
    button = ttk.Button(root, text = "submit", command =  dec_click).place(x=200,y=130)

# CBC Mode
def aes_cbc_encrypt(key, data, mode=AES.MODE_CBC):
    aes = AES.new(key, mode,IV)
    new_data = aes.encrypt(data)
    return new_data

def aes_cbc_decrypt(key, data, mode=AES.MODE_CBC):
    aes = AES.new(key, mode,IV)
    new_data = aes.decrypt(data)
    return new_data


class Example(Frame):
    def __init__(self, master, *pargs):
        Frame.__init__(self, master, *pargs)

        global filename

        self.image = Image.open(filename)
        self.img_copy= self.image.copy()


        self.background_image = ImageTk.PhotoImage(self.image)

        self.background = Label(self, image=self.background_image)
        self.background.pack(fill=BOTH, expand=YES)
        self.background.bind('<Configure>', self._resize_image)

    def _resize_image(self,event):

        new_width = event.width
        new_height = event.height

        self.image = self.img_copy.resize((new_width, new_height))

        self.background_image = ImageTk.PhotoImage(self.image)
        self.background.configure(image =  self.background_image)



def show():
          top=Toplevel()
          top.geometry('300x300')
          top.title(filename)
#           pic=Image.open(filename)  
#           resizedpic=pic.resize((250,250),Image.ANTIALIAS)
#           photo=ImageTk.PhotoImage(resizedpic)
#           label=Label(top,image=photo)
#           label.image=photo
#           label.pack()
          e=Example(top)
          e.pack(fill=BOTH,expand=YES)
        

btn = Button(root, text ='Open',width =12,command = open_file).place(x=200,y=30)
btn1 = Button(root, text ='Encrypt', command = get_enc_key).place(x=150,y=70)
btn2 = Button(root, text ='Decrypt', command = get_dec_key).place(x=250,y=70)

mainloop()
